<p align="center">
  <img width="550" height="281" src="http://encuentratubeca.mx/wp-content/uploads/2019/04/logoudgvectores_1_origsdaasss.png">
</p>

# <center>Proyecto Final</center>             
------
## Maestría en Ciencias de Datos
#### Alberto Martinez
#### Mauricio Tenorio

#### Marzo  2023

In [11]:
# Importamos librerias necesarias para el analisis de los datos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

#### Informacion del Dataset
Dataset utilizando 

Variables:

In [12]:
#Descargamos dataset directamente desde el url
path = "https://raw.githubusercontent.com/AlbertoMCD/ChatGPT_tweets/main/data/GPT.csv"
df = pd.read_csv(path)

In [13]:
df.head()

,text,favorited,favoriteCount,replyToSN,created,truncated,replyToSID,id,replyToUID,statusSource,screenName,retweetCount,isRetweet,retweeted,longitude,latitude
0,RT @gdb: ChatGPT as a tool for Congress: https...,False,0,NaN,4/27/2023 17:35:29,False,NaN,1651641228520194048,NaN,"<a href=""http://twitter.com/download/iphone"" r...",nos_ult,58,True,False,NaN,NaN
1,RT @gdb: ChatGPT as a tool for Congress: https...,False,0,NaN,4/27/2023 17:35:28,False,NaN,1651641223428317185,NaN,"<a href=""http://twitter.com/download/android"" ...",verobouvier,58,True,False,NaN,NaN
2,RT @moemicky18: So chatgpt failed an accountin...,False,0,NaN,4/27/2023 17:35:27,False,NaN,1651641221893308416,NaN,"<a href=""http://twitter.com/download/iphone"" r...",uzayrmug,2,True,False,NaN,NaN
3,RT @JeanPROFITbiz: Les gens gagnent des millie...,False,0,NaN,4/27/2023 17:35:27,False,NaN,1651641218059608094,NaN,"<a href=""http://twitter.com/download/iphone"" r...",SachaMery,359,True,False,NaN,NaN
4,"does chatgpt mean I, as a wet lab biologist wh...",False,0,NaN,4/27/2023 17:35:26,True,NaN,1651641214557364234,NaN,"<a href=""http://twitter.com/download/android"" ...",ruth_hook_,0,False,False,NaN,NaN


In [14]:
df.shape

(3000, 16)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   text           3000 non-null   object 
 1   favorited      3000 non-null   bool   
 2   favoriteCount  3000 non-null   int64  
 3   replyToSN      371 non-null    object 
 4   created        3000 non-null   object 
 5   truncated      3000 non-null   bool   
 6   replyToSID     360 non-null    float64
 7   id             3000 non-null   int64  
 8   replyToUID     371 non-null    float64
 9   statusSource   3000 non-null   object 
 10  screenName     3000 non-null   object 
 11  retweetCount   3000 non-null   int64  
 12  isRetweet      3000 non-null   bool   
 13  retweeted      3000 non-null   bool   
 14  longitude      2 non-null      float64
 15  latitude       2 non-null      float64
dtypes: bool(4), float64(4), int64(3), object(5)
memory usage: 293.1+ KB


### Data Cleaning

In [15]:
df.isnull().sum()

text                0
favorited           0
favoriteCount       0
replyToSN        2629
created             0
truncated           0
replyToSID       2640
id                  0
replyToUID       2629
statusSource        0
screenName          0
retweetCount        0
isRetweet           0
retweeted           0
longitude        2998
latitude         2998
dtype: int64

In [21]:
alt.Chart(df).mark_arc().encode(
    theta = "favorited",
    value = "favorited:N"
)

c:\Users\mauricit\AppData\Local\Programs\Python\Python311\Lib\site-packages\altair\utils\core.py:714: UserWarning: Unrecognized encoding channel 'value'
  warnings.warn("Unrecognized encoding channel '{}'".format(encoding))


SchemaValidationError: Invalid specification

        altair.vegalite.v4.schema.core.FacetedEncoding, validating 'additionalProperties'

        Additional properties are not allowed ('value' was unexpected)
        

alt.Chart(...)